In [1]:
!apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [2]:
!wget "https://github.com/zoekimm/475/blob/main/scripts/english-russian.txt.zip?raw=true"

--2021-12-10 11:09:57--  https://github.com/zoekimm/475/blob/main/scripts/english-russian.txt.zip?raw=true
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/zoekimm/475/raw/main/scripts/english-russian.txt.zip [following]
--2021-12-10 11:09:57--  https://github.com/zoekimm/475/raw/main/scripts/english-russian.txt.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/zoekimm/475/main/scripts/english-russian.txt.zip [following]
--2021-12-10 11:09:57--  https://raw.githubusercontent.com/zoekimm/475/main/scripts/english-russian.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... conn

In [3]:
!unzip /content/english-russian.txt.zip?raw=true

Archive:  /content/english-russian.txt.zip?raw=true
replace english-russian.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: english-russian.txt     
replace __MACOSX/._english-russian.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/._english-russian.txt  


In [4]:
input = [] #English
output = [] #Russian

with open('./english-russian.txt', encoding='utf8') as f:
    for line in f:
        sentences = line.split('\t')[:2]
        input.append(sentences[1])
        output.append(sentences[0])

In [5]:
import string

#remove punctuations
input = [''.join(word for word in sentence if word not in string.punctuation) for sentence in input]
output = [''.join(word for word in sentence if word not in string.punctuation) for sentence in output]

#need to add SOS EOS token

In [6]:
from collections import Counter
from itertools import chain
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

class Vocab:
  def __init__(self, li): 
      #li = list of sentencs
        #self.language = language #eng OR rus
        self.li = li
        self.tokenizer = Tokenizer()
        self.listOfVocab = self.get_vocab()
        self.word2count = self.get_word2count()
        self.word2index = self.get_word2index()
        self.index2word = self.get_index2word()
        self.max_length = max([len(sentence.split()) for sentence in self.li])
        self.encoded = self.get_encoded()

  def get_vocab(self): 
    temp = []
    for i in self.li:
      temp.append(i.split())
    return list(chain(*temp)) #list of vocab

  def get_word2count(self):
    return Counter(self.listOfVocab)

  def get_word2index(self):
    self.tokenizer.fit_on_texts(self.li)
    return self.tokenizer.word_index

  def get_index2word(self):
    return self.tokenizer.index_word

  def get_encoded(self):
    sequence = self.tokenizer.texts_to_sequences(self.li)
    sequence = pad_sequences(sequence, self.max_length, padding='post')
    return sequence

In [7]:
#teacher forcing

output_in = []
output_target = []
for i in output:
  output_in.append('<SOS> ' + i)
  output_target.append(i + ' <EOS>')
  
rus_vocab = Vocab(input)
eng_vocab_in = Vocab(output_in)
eng_vocab_target = Vocab(output_target)

In [17]:
rus_vocab_size = len(rus_vocab.word2count)
eng_vocab_size = len(eng_vocab_target.word2count)

print(rus_vocab_size)
print(eng_vocab_size)

59814
18829


In [19]:
import tensorflow as tf

hidden_dim = 256 
embedding_size = 256 

class Encoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embededding_layer = tf.keras.layers.Embedding(rus_vocab_size, embedding_size)
        self.lstm = tf.keras.layers.LSTM(hidden_dim, return_sequences=False, return_state=True)
        
    def call(self, x):
        output = self.embededding_layer(x)
        _, state_h, state_c = self.lstm(output)
        states = (state_h, state_c)
        return states

class Decoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embededding_layer = tf.keras.layers.Embedding(eng_vocab_size, embedding_size)
        self.lstm = tf.keras.layers.LSTM(hidden_dim, return_sequences=True, return_state=True)
        self.dense_layer = tf.keras.layers.Dense(eng_vocab_size, activation='softmax')
        
    def call(self, x, initial_state):
        output = self.embededding_layer(x)
        output, state_h, state_c = self.lstm(output, initial_state = initial_state)
        output = self.dense_layer(output)
        states = (state_h, state_c)
        return output, states

In [20]:
encoder = Encoder()
encoder_inputs = tf.keras.layers.Input(shape=(None,))
encoder_states = encoder(encoder_inputs)

decoder = Decoder()
decoder_inputs = tf.keras.layers.Input(shape=(None,))
decoder_outputs, _ = decoder(decoder_inputs, initial_state = encoder_states)

model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 encoder_1 (Encoder)            ((None, 256),        15837696    ['input_2[0][0]']                
                                 (None, 256))                                                     
                                                                                                  
 decoder (Decoder)              ((None, None, 18829  10184589    ['input_3[0][0]',            

In [15]:
batch_size = 128
epochs = 50


encoder_input_arr = rus_vocab.get_encoded()
decoder_input_arr = eng_vocab_in.get_encoded()
decoder_target_arr = eng_vocab_target.get_encoded()
print(encoder_input_arr.shape)
print(decoder_input_arr.shape)
print(decoder_target_arr.shape)

(363386, 80)
(363386, 102)
(363386, 102)


In [16]:
model.compile(optimizer = "rmsprop", loss = "categorical_crossentropy")
model.fit([encoder_input_arr, decoder_input_arr], decoder_target_arr, batch_size = batch_size, epochs = epochs)

Epoch 1/50


ValueError: ignored

In [82]:
#import numpy as np

#encoder_input = np.reshape(encoder_input, (encoder_input.shape[0], 1, encoder_input.shape[1]))

#decoder_input = np.reshape(decoder_input, (decoder_input.shape[0], 1, decoder_input.shape[1]))

#decoder_target = np.reshape(decoder_target, (decoder_target.shape[0], 1, decoder_target.shape[1]))

In [ ]:
import torch
import torch.nn as nn

class Encoder(nn.Module): # shape (batch_size, maxlength)
    def __init__(self, input_size, hidden_size, embedding_size = 50, num_layers)):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, 0.5)
        self.hidden = hidden_size

    def forward(self, x):
        x = self.embedding(x)
        output, (h_n, c_n) = self.rnn(x)
        return output, (h_n, c_n)

class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, 0.5)
        self.hidden = hidden_size

class Seq2SeqModel(nn.Module):
    def __init__(self, encoder, decoder, hidden_size, num_class):
        super(Seq2SeqModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.fc = nn.Linear(hidden_size, num_class)

In [ ]:
n_decoder_tokens = len(rus_vocab.word2count)

def batch_generator(X, y, batch_size):
  while True:
    for i in range(len(X)//batch_size):
      encoder_input = np.zeros(batch_size, eng_vocab.max_length)
      decoder_input = np.zeros(batch_size, rus_vocab.max_length)
      target = np.zeros(batch_size, rus_vocab.max_length, n_decoder_tokens)
      for j in range(batch_size):
        index = i * batch_size + j
        if index >= len(X):
          break
        input_text = X[index]
        target_text = y[index]
        for k, word in enumerate(input_text.split()):
          encoder_input[j, k] = eng_vocab.word2index[word]
        for k, word in enumerate(target_text.split()):
          if k < len(target_text.split()) - 1:
            decoder_input[j, k] = rus_vocab.word2index[word]
          if k > 0:
            target[j, k-1, rus_vocab.word2index[word]] = 1

      yield ([encoder_input, decoder_input], target)

In [ ]:
def train(model)